In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.image as mpimg

np.set_printoptions(suppress=True)

In [2]:
# Makes a matrix homogeneous - appends one to the end of each row
def make_homo(no_homo):
    homo = np.hstack([no_homo, np.ones(no_homo.shape[0]).reshape(-1,1)])
    return homo

In [3]:
def DLT(x, X):
    # Txyz, X = Normalization(3, X)
    # Tuv, x = Normalization(2, x)
    
    A = []
    for i in range(x.shape[0]):
        A.append(
            [X[i][0], X[i][1], X[i][2], X[i][3],       0,       0,       0,       0, -x[i][0]*X[i][0], -x[i][0]*X[i][1], -x[i][0]*X[i][2], -x[i][0]]
            )
        A.append(
            [      0,       0,       0,       0, X[i][0], X[i][1], X[i][2], X[i][3], -x[i][1]*X[i][0], -x[i][1]*X[i][1], -x[i][1]*X[i][2], -x[i][1]]
            )
    
    A = np.asarray(A) * -1

    U,S,V = np.linalg.svd(A)
    
    # The column of V is the solution corresponding to the smallest singular value
    P = V[np.argmin(S)]

    # Reshape P to 3x4
    P = P.reshape(3,4)
    
    # Normalize
    # P = P/P[2, 3]


    return P

In [4]:
def project_points(P, X):
    x = P @ X.T
    x = x/x[2]
    return x.T[:, :2]

def get_projection_error(x, x_proj):
    return np.mean(np.linalg.norm(x_proj - x, axis=1))

In [5]:
def get_properties(P):
    KR = P[:, :3]
    Kt = P[:, 3]

    K, R  = np.linalg.qr(KR)
    
    K = K/K[2, 2]
    K_inv = np.linalg.pinv(K)

    t = K_inv @ Kt
    return K, R, t

In [6]:
def generate_projection_matrix(K, R, t):
    E = np.hstack([R, np.atleast_2d(t).T])
    I = K
    
    P = I @ E 
    return P

In [7]:
image_coords= np.array([
        (982,2046),  (986,2228),  (1004,2396), (1009,2586), (1017,2745),
        (1925,2015), (1920,2192), (1920,2369), (1920,2541), (1916,2705), 
        (1287,2161), (1292,2316), (1292,2484), 
        (1783,2488), (1787,2329), (1787,2161), 
        (1544,2091), (1544,2245), (1544,2400), (1548,2555)
    ])

world_coords = np.array([
                    [0,2,4], [0,3,4], [0,4,4], [0,5,4], [0,6,4],
                    [3,2,0], [3,3,0], [3,4,0], [3,5,0], [3,6,0],
                    [0,3,2], [0,4,2], [0,5,2],
                    [2,5,0], [2,4,0], [2,3,0],
                    [0,3,0], [0,4,0], [0,5,0], [0,6,0]
                ])

In [8]:
def RansacDLT(image_coords, world_coords, n_samples, thresh, n_iters):
    # small x denotes the image coordinates
    x = make_homo(image_coords)

    # big X denotes the world coordinates
    X = make_homo(world_coords)

    total_pts = x.shape[0]

    best_inliers = []
    P_best = None
    projection_error = None
    for i in range(n_iters):
        sample_idxs = np.random.randint(0, total_pts, n_samples)

        x_sampled = x[sample_idxs]
        X_sampled = X[sample_idxs]

        P = DLT(x_sampled, X_sampled)

        x_proj = make_homo(project_points(P, X))

        errors = np.linalg.norm(x_proj - x, axis = 1)

        inliers = np.where(errors < thresh)[0]
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            P_best = P
            projection_error = get_projection_error(x, x_proj)
    return P_best, best_inliers, projection_error

In [9]:
P, inliers, projection_error = RansacDLT(image_coords, world_coords, n_samples = 6, thresh = 5, n_iters = 10000)

inliers.size

/tmp/ipykernel_94723/2015651045.py:3: RuntimeWarning: invalid value encountered in divide
  x = x/x[2]
/tmp/ipykernel_94723/2015651045.py:3: RuntimeWarning: divide by zero encountered in divide
  x = x/x[2]


16

In [10]:
projection_error

3.526884426722794